In [1]:
import re
import pandas as pd

In [2]:
df = pd.read_csv(r'D:\Yeshiva_Univ\Courses\Capstone_project\CEOs_Culture_project\Data\ceo_information\CEOs_by_google_v2(Mar_14_2020).csv')


Rename the unnamed column as index

In [3]:
df.rename(columns={'Unnamed: 0' :'index'}, inplace = True)

Clear rows. Select the rows having values only

In [4]:
#Select the rows with values only to proceed RegEx
df_n = df[df.ceo.notnull()]

df_nv = df_n[(df_n['value'] != 'Null')]


Splite the 'value' column using regex

Succeed to split the values. However, only some of the should be selected: Born, Nationality, and Education

In [5]:
# For Born
df_nv['Born'] = df_nv['value'].str.extract(r"Born: (.+?)\'\," )

# For Nationality 
df_nv['Nationality'] = df_nv['value'].str.extract(r"Nationality: (.+?)\'\," )


# For Education
## Select the strings end with "', '" and the strings end with nothing
Education = df_nv['value'].str.extract(r"Education: (.+?)\'\,\s\'|Education: (.+)?")

## merge the extracted two columes
df_nv['Education'] = pd.concat([Education[0].dropna(), Education[1].dropna()]).reindex_like(Education)

#See the full contents of the rows of value
pd.set_option('display.max_colwidth', -1)


C:\Users\Jeeho\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Jeeho\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Jeeho\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

Extract only Birth of Year only as BoY

In [6]:
df_nv['BoY'] = df_nv['Born'].str.extract(pat = "([0-9]{4})" )

C:\Users\Jeeho\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Extract Nationality strings from Born based on 1) the postal abbreviations of the US and 2) the country names

In [7]:
# Create a list of the abbr

us_postal_abbr = {'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 
                  'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 
                  'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 
                  'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 
                  'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WY'}

reorder the index to run the for loop

In [8]:
df_nv_index = pd.Series(range(len(df_nv))) 

In [9]:
df_nv = df_nv.set_index([df_nv_index])

In [10]:
# Create a column for the extracted us abbreviation code 
df_nv['us_abbr'] = df_nv['Born'].str.extract(pat = "([A-Z]{2})")

# Insert value 'American' in the 'Nationality' column if where the us_abbr matchs with the list of abbr
df_nv.loc[df_nv['us_abbr'].isin(us_postal_abbr), 'Nationality'] = 'American'

For nationalities

In [11]:
pd.set_option('display.max_rows', None)

Catch the missing countries among the values

In [12]:
#Extract the United States and United Kingdom and the other country names
bn = df_nv['Born'].str.extract(".*?([A-Za-z]{6}\s[A-Za-z]{6,7})$|.*?([A-Za-z]{5,11})$")

## merge the extracted two columes
df_nv['born_country'] = pd.concat([bn[0].dropna(), bn[1].dropna()]).reindex_like(bn)

#Replacing the NaN values of Nationality column with newly extracted values 
df_nv['Nationality'].fillna(df_nv['born_country'], inplace=True)

In [13]:
display(df_nv)

,index,ceo,value,Born,Nationality,Education,BoY,us_abbr,born_country
0,0,Doug McMillon,"Born: October 17, 1966 (age 53 years), Memphis, TN', 'Nationality: American', 'Spouse: Shelley McMillon', 'Education: Bentonville High School, Sam M. Walton College of Business, The University of Tulsa, University of Arkansas', 'Children: Blake McMillon, Spencer McMillon', 'Parents: Laura McMillon, Morris McMillon","October 17, 1966 (age 53 years), Memphis, TN",American,"Bentonville High School, Sam M. Walton College of Business, The University of Tulsa, University of Arkansas",1966,TN,NaN
1,2,Ben van Beurden,"Born: April 23, 1958 (age 61 years), Netherlands', 'Full name: Bernardus Cornelis Adriana Margriet van Beurden', 'Nationality: Dutch', 'Spouse: Stacey van Beurden', 'Children: 4', 'Term: January 2014–present","April 23, 1958 (age 61 years), Netherlands",Dutch,NaN,1958,NaN,Netherlands
2,5,Amin H. Nasser,"Born: 1960 (age 60 years)', 'Nationality: Saudi', 'Education: جامعة الملك فهد للبترول والمعادن",1960 (age 60 years),Saudi,جامعة الملك فهد للبترول والمعادن,1960,NaN,NaN
3,6,Bob Dudley,"Born: September 14, 1955 (age 64 years), Queens, New York, NY', 'Nationality: American', 'Spouse: Mary Dudley', 'Citizenship: United Kingdom; United States', 'Title: Former Group Chief Executive of BP', 'Education: University of Illinois at Urbana-Champaign, MORE","September 14, 1955 (age 64 years), Queens, New York, NY",American,"University of Illinois at Urbana-Champaign, MORE",1955,NY,NaN
4,7,Darren Woods,"Born: Wichita, KS', 'Spouse: Kathryn Woods', 'Predecessor: Rex W. Tillerson', 'Education: Kellogg School of Management, Texas A&M University","Wichita, KS",American,"Kellogg School of Management, Texas A&M University",NaN,KS,NaN
5,8,Herbert Diess,"Born: October 24, 1958 (age 61 years), Munich, Germany', 'Education: Technische Universität München (1987), MORE', 'Board member of: Audi, Volkswagen Group, SEAT, Škoda Auto, Porsche Holding","October 24, 1958 (age 61 years), Munich, Germany",Germany,"Technische Universität München (1987), MORE",1958,NaN,Germany
6,9,Akio Toyoda,"Born: May 3, 1956 (age 63 years), Nagoya, Aichi, Japan', 'Nationality: Japanese', 'Children: 2', 'Education: Babson College, Keio University', 'Parents: Shoichiro Toyoda, Hiroko Mitsui', 'Grandparents: Kiichiro Toyoda, Hatako Toyoda","May 3, 1956 (age 63 years), Nagoya, Aichi, Japan",Japanese,"Babson College, Keio University",1956,NaN,Japan
7,10,Tim Cook,"Born: November 1, 1960 (age 59 years), Mobile, AL', 'Height: 6′ 0″', 'Net worth: US$ 1.3 billion (2015)', 'Education: The Fuqua School of Business (1988), MORE', 'Parents: Geraldine Cook, Donald Cook', 'Siblings: Gerald Cook, Michael Cook","November 1, 1960 (age 59 years), Mobile, AL",American,"The Fuqua School of Business (1988), MORE",1960,AL,NaN
8,11,Warren Buffett,"Born: August 30, 1930 (age 89 years), Omaha, NE', 'Net worth: 73.4 billion USD (2020)', 'Education: Columbia Business School (1950–1951), MORE', 'Spouse: Astrid Menks (m. 2006), Susan Buffett (m. 1952–2004)","August 30, 1930 (age 89 years), Omaha, NE",American,"Columbia Business School (1950–1951), MORE",1930,NE,NaN
9,12,Jeff Bezos,"Net worth: 106.2 billion USD (2020) Trending', 'Born: January 12, 1964 (age 56 years), Albuquerque, NM', 'Height: 5′ 7″', 'Nationality: American', 'Children: 4', 'Education: Princeton University (1986), River Oaks Elementary School, Miami Palmetto Senior High School, University of Florida","January 12, 1964 (age 56 years), Albuquerque, NM",American,"Princeton University (1986), River Oaks Elementary School, Miami Palmetto Senior High School, University of Florida",1964,NM,NaN


In [15]:
df_nv.to_csv(r'D:\Yeshiva_Univ\Courses\Capstone_project\CEOs_Culture_project\Data\ceo_information\CEOs_info_v1(Apr_1_2020).csv')